# Introduction

This tutorial is about linear transforms - a basic building block of neural networks, including deep learning models.

# Virtual environments and syncing repositories

Before you proceed onwards, remember to activate you virtual environments so you can use the software you installed last week as well as run the notebooks in interactive mode, not through the github.com website.

## Virtual environments

To activate the virtual environment:
   * If you were in last week's Tuesday or Wednesday group type `activate_mlp` or `source ~/mlpractical/venv/bin/activate`
   * If you were in the Monday group:
      + and if you have chosen the **comfy** way type: `workon mlpractical`
      + and if you have chosen the **generic** way, `source` your virutal environment using `source` and specyfing the path to the activate script (you need to localise it yourself, there were not any general recommendations w.r.t dir structure and people have installed it in different places, usually somewhere in the home directories. If you cannot easily find it by yourself, use something like: `find . -iname activate` ):

## On Synchronising repositories

Enter the git mlp repository you set up last week (i.e. `~/mlpractical/repo-mlp`) and once you sync the repository (in one of the two below ways), start the notebook session by typing:

```
ipython notebook
```

### Default way

To avoid potential conflicts between the changes you have made since last week and our additions, we recommend `stash` your changes and `pull` the new code from the mlpractical repository by typing:

1. `git stash save "Lab1 work"`
2. `git pull`

Then, if you need to, you can always (temporaily) restore a desired state of the repository.

### For advanced github users

It is OK if you want to keep your changes and merge the new code with whatever you already have, but you need to know what you are doing and how to resolve conflicts.
 

# Single Layer Models

***
### Note on storing matrices in computer memory

Suppose you want to store the following matrix in memory: $\left[ \begin{array}{ccc}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9 \end{array} \right]$ 

If you allocate the memory at once for the whole matrix, then the above matrix would be organised as a vector in one of two possible forms:

* Row-wise layout where the order would look like: $\left [ \begin{array}{ccccccccc}
1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \end{array} \right ]$
* Column-wise layout where the order would look like: $\left [ \begin{array}{ccccccccc}
1 & 4 & 7 & 2 & 5 & 8 & 3 & 6 & 9 \end{array} \right ]$

Although `numpy` can easily handle both formats (possibly with some computational overhead), in our code we will stick with the more modern (and default) `C`-like approach and use the row-wise format (in contrast to Fortran that used a column-wise approach). 

This means, that in this tutorial:
* vectors are kept row-wise $\mathbf{x} = (x_1, x_1, \ldots, x_D) $ (rather than $\mathbf{x} = (x_1, x_1, \ldots, x_D)^T$)
* similarly, in case of matrices we will stick to: $\left[ \begin{array}{cccc}
x_{11} & x_{12} & \ldots & x_{1D} \\
x_{21} & x_{22} & \ldots & x_{2D} \\
x_{31} & x_{32} & \ldots & x_{3D} \\ \end{array} \right]$ and each row (i.e. $\left[ \begin{array}{cccc} x_{11} & x_{12} & \ldots & x_{1D} \end{array} \right]$) represents a single data-point (like one MNIST image or one window of observations)

In lecture slides you will find the equations following the conventional mathematical approach, using column vectors, but you can easily map between column-major and row-major organisations using a matrix transpose.

***

## Linear and Affine Transforms

The basis of all linear models is the so called affine transform, which is a transform that implements a linear transformation and translation of the input features. The transforms we are going to use are parameterised by:

  * A weight matrix $\mathbf{W} \in \mathbb{R}^{D\times K}$: where element $w_{ik}$ is the weight from input $x_i$ to output $y_k$
  * A bias vector $\mathbf{b}\in R^{K}$ : where element $b_{k}$ is the bias for output $k$

Note, the bias is simply some additive term, and can be easily incorporated into an additional row in weight matrix and an additional input in the inputs which is set to $1.0$ (as in the below picture taken from the lecture slides). However, here (and in the code) we will keep them separate.

![Making Predictions](res/singleLayerNetWts-1.png)

For instance, for the above example of 5-dimensional input vector by $\mathbf{x} = (x_1, x_2, x_3, x_4, x_5)$, weight matrix $\mathbf{W}=\left[ \begin{array}{ccc}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} \\
w_{31} & w_{32} & w_{33} \\
w_{41} & w_{42} & w_{43} \\
w_{51} & w_{52} & w_{53} \\ \end{array} \right]$, bias vector $\mathbf{b} = (b_1, b_2, b_3)$ and outputs $\mathbf{y} = (y_1, y_2, y_3)$, one can write the transformation as follows:

(for the $i$-th output)

(1) $
\begin{equation}
   y_i = b_i + \sum_j x_jw_{ji}
\end{equation}
$

or the equivalent vector form (where $\mathbf w_i$ is the $i$-th row of $\mathbf W$):

(2) $
\begin{equation}
   y_i = b_i + \mathbf x \mathbf w_i^T
\end{equation}
$

The same operation can be also written in matrix form, to compute all the outputs $\mathbf{y}$ at the same time:

(3) $
\begin{equation}
  \mathbf y=\mathbf x\mathbf W + \mathbf b
\end{equation}
$

This is equivalent to slides 12/13 in lecture 1, except we are using row vectors.

When $\mathbf{x}$ is a mini-batch (contains $B$ data-points of dimension $D$ each), i.e. $\left[ \begin{array}{cccc}
x_{11} & x_{12} & \ldots & x_{1D} \\
x_{21} & x_{22} & \ldots & x_{2D} \\
\cdots \\
x_{B1} & x_{B2} & \ldots & x_{BD} \\ \end{array} \right]$ equation (3) effectively becomes to be

(4) $
\begin{equation}
  \mathbf Y=\mathbf X\mathbf W + \mathbf b
\end{equation}
$

where both $\mathbf{X}\in\mathbb{R}^{B\times D}$ and $\mathbf{Y}\in\mathbb{R}^{B\times K}$ are matrices, and $\mathbf{b}$ needs to be <a href="http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html">broadcasted</a> $B$ times (numpy will do this by default). However, we will not make an explicit distinction between a special case for $B=1$ and $B>1$ and simply use equation (3) instead, although $\mathbf{x}$ and hence $\mathbf{y}$ could be matrices. From an implementation point of view, it does not matter.

The desired functionality for matrix multiplication in numpy is provided by <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html">numpy.dot</a> function. If you haven't use it so far, get familiar with it as we will use it extensively.

In [34]:
# Dot products
import numpy as np
a = np.array([1,2,3])
b = np.array([3,2,1])
print "The dot product of a and b is", np.dot(a,b)
print "The element-wise product of a and b is", a*b

The dot product of a and b is 10
The element-wise product of a and b is [3 4 3]


### A general note on random number generators

It is generally a good practice (for machine learning applications **not** for cryptography!) to seed a pseudo-random number generator once at the beginning of the experiment, and use it later through the code where necesarry. This makes it easier to reproduce results since random initialisations can be replicated. As such, within this course we are going use a single random generator object, similar to the below:

In [35]:
#initialise the random generator to be used later
seed=[2015, 10, 1]
random_generator = numpy.random.RandomState(seed)

## Exercise 1 

Using `numpy.dot`, implement **forward** propagation through the linear transform defined by equations (3) and (4) for $B=1$ and $B>1$. As data ($\mathbf{x}$) use `MNISTDataProvider` introduced last week. For the case when $B=1$, write a function to compute the 1st output ($y_1$) using equations (1) and (2). Check if the output is the same as the corresponding one obtained with numpy. 

Tip: To generate random data you can use `random_generator.uniform(-0.1, 0.1, (D, 10))` from above.

In [36]:
from mlp.dataset import MNISTDataProvider

mnist_dp = MNISTDataProvider(dset='valid', batch_size=3, max_num_batches=1, randomize=False)

irange = 0.1
W = random_generator.uniform(-irange, irange, (784,10)) 
b = numpy.zeros((10,))


In [37]:
mnist_dp.reset()

#implement following functions, then run the cell
def y1_equation_1(x, W, b):
    y1 = b[0]
    for j in range(len(x)):
        y1 += x[j]*W[j,0]
    return y1
    
def y1_equation_2(x, W, b):
    y1 = b[0] + np.dot(x, W[:,0])
    return y1

def y_equation_3(x, W, b):
    #use numpy.dot
    y = np.zeros((np.shape(x)[0], np.shape(W)[1]))
    for j in range(np.shape(x)[0]): # Each data point separately
        y[j,:] = np.dot(x[j,:],W) + b
    return y

def y_equation_4(x, W, b):
    #use numpy.dot
    y = np.dot(x,W) + b # All together
    return y

for x, t in mnist_dp:
    y1e1 = y1_equation_1(x[0], W, b)
    y1e2 = y1_equation_2(x[0], W, b)
    ye3 = y_equation_3(x, W, b)
    ye4 = y_equation_4(x, W, b)

print 'y1e1', y1e1
print 'y1e1', y1e1
print 'ye3', ye3
print 'ye4', ye4
    

y1e1 -0.0442595171786
y1e1 -0.0442595171786
ye3 [[-0.04  0.92 -0.4   0.94  0.2  -0.12  0.67 -0.04 -0.22 -0.04]
 [ 0.3   0.43 -0.57 -0.29 -0.89 -0.16  0.64  0.39 -0.24 -0.16]
 [ 0.07  0.05 -0.11 -0.5  -0.2  -0.23  0.18  0.81 -0.48  0.05]]
ye4 [[-0.04  0.92 -0.4   0.94  0.2  -0.12  0.67 -0.04 -0.22 -0.04]
 [ 0.3   0.43 -0.57 -0.29 -0.89 -0.16  0.64  0.39 -0.24 -0.16]
 [ 0.07  0.05 -0.11 -0.5  -0.2  -0.23  0.18  0.81 -0.48  0.05]]


In [38]:
print mnist_dp._curr_idx
mnist_dp.reset()
x, t = mnist_dp.next() # This is how we get access to a single batch (the current one)
print np.shape(x), '\n', np.shape(t), '\n'
print mnist_dp._curr_idx


9999
(3, 784) 
(3, 10) 

3


## Exercise 2

Modify the examples from Exercise 1 to perform **backward** propagation, that is, given $\mathbf{y}$ (obtained in the previous step) and weight matrix $\mathbf{W}$, project $\mathbf{y}$ onto the input space $\mathbf{x}$ (ignore or set to zero the biases towards $\mathbf{x}$ in backward pass). Mathematically, we are interested in the following transformation: $\mathbf{z}=\mathbf{y}\mathbf{W}^T$

In [39]:
def x_equation_bck(y, W):
    x = np.dot(y, W.T)
    return x

In [40]:
x_star = x_equation_bck(y1e1,W)

In [41]:
print x_star[1:10]

[[-0.  0. -0. ..., -0.  0.  0.]
 [-0. -0.  0. ..., -0. -0. -0.]
 [ 0. -0.  0. ..., -0. -0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -0.]
 [-0. -0. -0. ...,  0.  0.  0.]
 [-0.  0.  0. ...,  0. -0.  0.]]


***
## Exercise 3 (optional)

In case you do not fully understand how matrix-vector and/or matrix-matrix products work, consider implementing `my_dot_mat_mat` function  (you have been given `my_dot_vec_mat` code to look at as an example) which takes as the input the following arguments:

* D-dimensional input vector $\mathbf{x} = (x_1, x_2, \ldots, x_D) $.
* Weight matrix $\mathbf{W}\in\mathbb{R}^{D\times K}$:

and returns:

* K-dimensional output vector $\mathbf{y} = (y_1, \ldots, y_K) $

Your job is to write a variant that works in a mini-batch mode where both $\mathbf{x}\in\mathbb{R}^{B\times D}$ and $\mathbf{y}\in\mathbb{R}^{B\times K}$ are matrices in which each rows contain one of $B$ data-points from mini-batch (rather than  $\mathbf{x}\in\mathbb{R}^{D}$ and $\mathbf{y}\in\mathbb{R}^{K}$).

In [42]:
def my_dot_vec_mat(x, W):
    D = x.shape[0]
    K = W.shape[1]
    assert (D == W.shape[0]), (
        "Number of columns of x expected to "
        " to be equal to the number of rows in "
        "W, but got shapes %s, %s" % (x.shape, W.shape)
    )
    y = numpy.zeros((K,))
    for k in xrange(0, K):
        for d in xrange(0, D):
            y[k] += x[d] * W[d,k]
                
    return y

In [43]:
irange = 0.1 #+-range from which we draw the random numbers

x = random_generator.uniform(-irange, irange, (5,)) 
W = random_generator.uniform(-irange, irange, (5,3)) 

y_my = my_dot_vec_mat(x, W)
y_np = numpy.dot(x, W)

same = numpy.allclose(y_my, y_np)

if same:
    print 'Well done!'
else:
    print 'Matrices are different:'
    print 'y_my is: ', y_my
    print 'y_np is: ', y_np

Well done!


In [44]:
def my_dot_mat_mat(x, W):
    B = x.shape[0]
    D = x.shape[1]
    K = W.shape[1]
    assert (D == W.shape[0]), (
        "Number of columns in of x expected to "
        " to be the same as rows in W, but got shapes %s %s" % (x.shape, W.shape)
    )
    #allocate the output container
    y = numpy.zeros((B, K))
    
    #implement here matrix-matrix inner product here
    for b in xrange(B):
        for k in xrange(K):
            for d in range(D):
                y[b,k] += x[b,d]*W[d,k]
                
    return y

Test whether you get comparable numbers to what numpy is producing:

In [45]:
irange = 0.1 #+-range from which we draw the random numbers

x = random_generator.uniform(-irange, irange, (2,5)) 
W = random_generator.uniform(-irange, irange, (5,3)) 

y_my = my_dot_mat_mat(x, W)
y_np = numpy.dot(x, W)

same = numpy.allclose(y_my, y_np)

if same:
    print 'Well done!'
else:
    print 'Matrices are different:'
    print 'y_my is: ', y_my
    print 'y_np is: ', y_np

Well done!


Now we benchmark each approach (we do it in separate cells, as timeit currently can measure whole cell execuiton only).

In [46]:
#generate bit bigger matrices, to better evaluate timings
x = random_generator.uniform(-irange, irange, (10, 1000))
W = random_generator.uniform(-irange, irange, (1000, 100))

In [47]:
print 'my_dot timings:'
%timeit -n10 my_dot_mat_mat(x, W)

my_dot timings:
10 loops, best of 3: 744 ms per loop


In [75]:
print 'numpy.dot timings:'
%timeit -n10 numpy.dot(x, W)

numpy.dot timings:
10 loops, best of 3: 159 µs per loop


**Optional section ends here**
***

# Iterative learning of linear models

We will learn the model with stochastic gradient descent on N data-points using mean square error (MSE) loss function, which is defined as follows:

(5) $ \normalsize
E = \frac{1}{2} \sum_{n=1}^N ||\mathbf{y}^n - \mathbf{t}^n||^2 =  \sum_{n=1}^N E^n \\
  E^n = \frac{1}{2} ||\mathbf{y}^n - \mathbf{t}^n||^2
$

(6) $ \normalsize E^n = \frac{1}{2} \sum_{k=1}^K (y_k^n - t_k^n)^2 $
  
Hence, the gradient w.r.t (with respect to) the $r$ output y of the model is defined as, so called delta function, $\delta_r$: 

(8) $\normalsize \frac{\partial{E^n}}{\partial{y_{r}}} = (y^n_r - t^n_r) =  \delta^n_r \quad ; \quad
    \delta^n_r = y^n_r - t^n_r \\
    \frac{\partial{E}}{\partial{y_{r}}} = \sum_{n=1}^N \frac{\partial{E^n}}{\partial{y_{r}}} = \sum_{n=1}^N \delta^n_r
$

Also: $ \frac{\partial{y_r^n}}{\partial{w_{sr}}} = x_s^n$

Similarly, using the above $\delta^n_r$ one can express the gradient of the  weight $w_{sr}$ (from the s-th input to the r-th output) for linear model and MSE cost as follows:

(9) $ \normalsize
    \frac{\partial{E^n}}{\partial{w_{sr}}} = \frac{\partial{E^n}}{\partial{y_{r}^n}} \frac{\partial{y_r^n}}{\partial{w_{sr}}} = (y^n_r - t^n_r)x_s^n =  \delta^n_r x_s^n \quad\\
    \frac{\partial{E}}{\partial{w_{sr}}} = \sum_{n=1}^N \frac{\partial{E^n}}{\partial{w_{rs}}} = \sum_{n=1}^N \delta^n_r x_s^n
$

and the gradient for bias parameter at the $r$-th output is:

(10) $ \normalsize
    \frac{\partial{E}}{\partial{b_{r}}} = \sum_{n=1}^N \frac{\partial{E^n}}{\partial{b_{r}}} = \sum_{n=1}^N \delta^n_r
$ (since the corresponding $x_0$ is always equal to 1).


![Making Predictions](res/singleLayerNetPredict.png)
 
  * Input vector $\mathbf{x} = (x_1, x_2, \ldots, x_D) $
  * Output scalar $y_1$
  * Weight matrix $\mathbf{W}$: $w_{ik}$ is the weight from input $x_i$ to output $y_k$. Note, here this is really a vector since a single scalar output, y_1.
  * Scalar bias $b$ for the only output in our model 
  * Scalar target $t$ for the only output in out model
  
First, ensure you can make use of the data provider (note, for training data has been normalised to zero mean and unit variance, hence different effective range than one can find in file):

In [48]:
from mlp.dataset import MetOfficeDataProvider

modp = MetOfficeDataProvider(window_size = 10, batch_size=10, max_num_batches=2, randomize=False)

%precision 2
for x, t in modp:
    print 'Observations: \n', x
    print 'To predict: \n', t

Observations: 
[[-0.11  0.35  0.08 -0.13  0.39  0.13 -0.05 -0.07 -0.09 -0.12]
 [ 0.35  0.08 -0.13  0.39  0.13 -0.05 -0.07 -0.09 -0.12 -0.08]
 [ 0.08 -0.13  0.39  0.13 -0.05 -0.07 -0.09 -0.12 -0.08 -0.06]
 [-0.13  0.39  0.13 -0.05 -0.07 -0.09 -0.12 -0.08 -0.06 -0.04]
 [ 0.39  0.13 -0.05 -0.07 -0.09 -0.12 -0.08 -0.06 -0.04 -0.13]
 [ 0.13 -0.05 -0.07 -0.09 -0.12 -0.08 -0.06 -0.04 -0.13 -0.13]
 [-0.05 -0.07 -0.09 -0.12 -0.08 -0.06 -0.04 -0.13 -0.13 -0.13]
 [-0.07 -0.09 -0.12 -0.08 -0.06 -0.04 -0.13 -0.13 -0.13 -0.1 ]
 [-0.09 -0.12 -0.08 -0.06 -0.04 -0.13 -0.13 -0.13 -0.1   0.08]
 [-0.12 -0.08 -0.06 -0.04 -0.13 -0.13 -0.13 -0.1   0.08 -0.06]]
To predict: 
[[-0.08]
 [-0.06]
 [-0.04]
 [-0.13]
 [-0.13]
 [-0.13]
 [-0.1 ]
 [ 0.08]
 [-0.06]
 [ 0.04]]
Observations: 
[[-0.08 -0.06 -0.04 -0.13 -0.13 -0.13 -0.1   0.08 -0.06  0.04]
 [-0.06 -0.04 -0.13 -0.13 -0.13 -0.1   0.08 -0.06  0.04  0.12]
 [-0.04 -0.13 -0.13 -0.13 -0.1   0.08 -0.06  0.04  0.12 -0.1 ]
 [-0.13 -0.13 -0.13 -0.1   0.08 -0.06  0.04  0

## Exercise 4

The below code implements a very simple variant of stochastic gradient descent for the rainfall prediction example. Your task is to implement 5 functions in the next cell and then run two next cells that 1) build sgd functions and 2) run the actual training.

### Reminder:

B: number of examples in each batch

D: input dimensionality

K: output dimensionality

* $ X \in \mathbb{R}^{B\times D}$
* $W \in \mathbb{R}^{D\times K}$,
* $ Y \in \mathbb{R}^{B\times K}$,
* $ b \in \mathbb{R}^{K}$ (bias)

It might be the case that $B = 1$, so we need to take this into account.

In [204]:
#When implementing those, take into account the mini-batch case, for which one is
#expected to sum the errors for each example

def fprop(x, W, b):
    #code implementing eq. (3)
    # I need to fix the asserts to account for the case where B = 1 (by looking into ndim(x))
    assert (x.shape[1] == W.shape[0]), (
        "Number of columns in of x expected to "
        " to be the same as rows in W, but got shapes %s %s" % (x.shape, W.shape)
    )
    
    assert (b.shape[0] == W.shape[1]), (
        "Number of columns in of x expected to "
        " to be the same as rows in W, but got shapes %s %s" % (x.shape, W.shape)
    )
    
    y = np.dot(x,W) + b
    return y

def cost(y, t):
    #Mean Square Error cost, equation (5)
    assert (y.shape == t.shape), (
        "The vectos y and t are expected to have same shapes "
        " but got shapes %s %s" % (y.shape, t.shape)
    )
    cost_sum = np.sum(np.linalg.norm((y-t), axis = 0), axis = 0)
    
    return 0.5*cost_sum

            
def cost_grad(y, t):
    #Gradient of the cost w.r.t y equation (8)
    assert (y.shape == t.shape), (
        "The vectos y and t are expected to have same shapes "
        " but got shapes %s %s" % (y.shape, t.shape)
    )
    cost_gradient = y-t
    #cost_gradient = np.sum(y-t, axis = 0) # Total gradient
    return cost_gradient # B X K

def cost_wrt_W(cost_grad, x):
    #Gradient of the cost w.r.t W, equation (9)
    B, D = x.shape
    K = cost_grad.shape[1] # Gradient of E wrt y
    
    cost_gradient_w = np.zeros((D, K))
    for ss in xrange(D):
        for rr in xrange(K):
            cost_gradient_w[ss,rr] =  np.dot(x[:,ss],cost_grad[:,rr])
    
    return cost_gradient_w
        
def cost_wrt_b(cost_grad):
    #Gradient of the cost w.r.t to b, equation (10)
    K = cost_grad.shape[1] # Gradient of E wrt y
    cost_gradient_b = np.zeros(K)
    tmp = cost_grad
    for rr in xrange(K):
        cost_gradient_b[rr] = sum(tmp[:,rr])
    
    return cost_gradient_b


In [205]:
# Testing (1-dimensional output)
y = t + np.random.randn(y.shape[0],y.shape[1]) # B X 1
print 'Cost:', cost(y,t)
print 'Shape of y and t:', y.shape
print 'Gradient w.r.t y: \n', cost_grad(y,t)
print 'Gradient shape:', cost_grad(y,t).shape
print 'Gradient w.r.t W: \n', cost_wrt_W(cost_grad(y,t), x)
print 'Griadent shape:', cost_wrt_W(cost_grad(y,t), x).shape
print 'Gradient w.r.t b: \n', cost_wrt_b(cost_grad(y,t))
print 'Griadent shape:', cost_wrt_b(cost_grad(y,t)).shape

Cost: 1.86407968411
Shape of y and t: (10, 1)
Gradient w.r.t y: 
[[ 1.67]
 [-1.56]
 [ 1.38]
 [-0.75]
 [ 1.2 ]
 [ 0.7 ]
 [-1.03]
 [ 1.38]
 [ 0.21]
 [-1.12]]
Gradient shape: (10, 1)
Gradient w.r.t W: 
[[-0.1 ]
 [-0.62]
 [ 0.23]
 [ 0.17]
 [-0.51]
 [ 0.02]
 [-0.79]
 [ 0.39]
 [ 0.21]
 [-0.68]]
Griadent shape: (10, 1)
Gradient w.r.t b: 
[ 2.08]
Griadent shape: (1,)


In [206]:
# Testing (2-dimensional output, B X K)
t_qwe=np.hstack((t,t))
print 'Cost:', cost(y_qwe,t_qwe)
print 'Shape of y and t:', t_qwe.shape
y_qwe = t_qwe + np.random.randn(t_qwe.shape[0],t_qwe.shape[1])
print 'Gradient w.r.t y: \n', cost_grad(y_qwe,t_qwe)
print 'Gradient shape:', cost_grad(y_qwe,t_qwe).shape
print 'Gradient w.r.t W: \n', cost_wrt_W(cost_grad(y_qwe,t_qwe), x)
print 'Griadent shape:', cost_wrt_W(cost_grad(y_qwe,t_qwe), x).shape
print 'Gradient w.r.t b: \n', cost_wrt_b(cost_grad(y_qwe,t_qwe))
print 'Griadent shape:', cost_wrt_b(cost_grad(y_qwe,t_qwe)).shape

Cost: 2.91478562047
Shape of y and t: (10, 2)
Gradient w.r.t y: 
[[ 0.6  -0.3 ]
 [ 0.35 -0.73]
 [ 1.57 -0.11]
 [-1.31  1.6 ]
 [-0.59 -0.64]
 [-1.14  1.33]
 [ 0.43  0.38]
 [-1.02  1.81]
 [-0.76 -1.56]
 [ 0.41  0.51]]
Gradient shape: (10, 2)
Gradient w.r.t W: 
[[ 0.19  0.01]
 [ 0.23 -0.27]
 [-0.33 -0.11]
 [-0.24  0.17]
 [-0.15  0.39]
 [ 0.1  -0.01]
 [-0.04 -0.21]
 [-0.43 -0.5 ]
 [ 0.09  1.05]
 [ 0.06 -0.41]]
Griadent shape: (10, 2)
Gradient w.r.t b: 
[-1.46  2.28]
Griadent shape: (2,)


In [202]:
def sgd_epoch(data_provider, W, b, learning_rate):
    mse_stats = []
    
    #get the minibatch of data
    for x, t in data_provider:
        
        #1. get the estimate of y
        y = fprop(x, W, b)

        #2. compute the loss function
        tmp = cost(y, t)
        mse_stats.append(tmp)
        
        #3. compute the grad of the cost w.r.t the output layer activation y
        #i.e. how the cost changes when output y changes
        cost_grad_deltas = cost_grad(y, t)

        #4. compute the gradients w.r.t model's parameters
        grad_W = cost_wrt_W(cost_grad_deltas, x)
        grad_b = cost_wrt_b(cost_grad_deltas)

        #4. Update the model, we update with the mean gradient
        # over the minibatch, rather than sum of particular gradients
        # in a minibatch, to do so we scale the learning rate by batch_size
        batch_size = x.shape[0]
        effect_learn_rate = learning_rate / batch_size

        W = W - effect_learn_rate * grad_W
        b = b - effect_learn_rate * grad_b
    
    return W, b, numpy.mean(mse_stats)

def sgd(data_provider, W, b, learning_rate=0.1, max_epochs=10):
    
    for epoch in xrange(0, max_epochs):
        #reset the data provider
        data_provider.reset()
        
        #train for one epoch
        W, b, mean_cost = \
            sgd_epoch(data_provider, W, b, learning_rate)
                
        print "MSE training cost after %d-th epoch is %f" % (epoch + 1, mean_cost)
    
    return W, b
        
        

In [ ]:
#some hyper-parameters
window_size = 12
irange = 0.5
learning_rate = 0.01
max_epochs=40

# note, while developing you can set max_num_batches to some positive number to limit
# the number of training data-points (you will get feedback faster)
mdp = MetOfficeDataProvider(window_size, batch_size=10, max_num_batches=-100, randomize=False)

#initialise the parameters
W = random_generator.uniform(-irange, irange, (window_size, 1))
b = random_generator.uniform(-irange, irange, (1, ))

#train the model
sgd(mdp, W, b, learning_rate=learning_rate, max_epochs=max_epochs)


MSE training cost after 1-th epoch is 0.306069
MSE training cost after 2-th epoch is 0.255906
MSE training cost after 3-th epoch is 0.244427
MSE training cost after 4-th epoch is 0.241125
MSE training cost after 5-th epoch is 0.240180
MSE training cost after 6-th epoch is 0.239903
MSE training cost after 7-th epoch is 0.239818
MSE training cost after 8-th epoch is 0.239790
MSE training cost after 9-th epoch is 0.239779
MSE training cost after 10-th epoch is 0.239775
MSE training cost after 11-th epoch is 0.239773
MSE training cost after 12-th epoch is 0.239772
MSE training cost after 13-th epoch is 0.239771
MSE training cost after 14-th epoch is 0.239771
MSE training cost after 15-th epoch is 0.239771
MSE training cost after 16-th epoch is 0.239771
MSE training cost after 17-th epoch is 0.239771
MSE training cost after 18-th epoch is 0.239771
MSE training cost after 19-th epoch is 0.239771
MSE training cost after 20-th epoch is 0.239771
MSE training cost after 21-th epoch is 0.239771
M

## Exercise 5

Modify the above prediction (regression) problem so the model makes a binary classification whether the the weather is going to be one of those \{rainy, not-rainy} (look at slide 12 of the 2nd lecture)

Tip: You need to introduce the following changes:
1. Modify `MetOfficeDataProvider` (for example, inherit from MetOfficeDataProvider to create a new class MetOfficeDataProviderBin) and modify `next()` function so it returns as `targets` either 0 (not-rainy - if the the amount of rain [before mean/variance normalisation] is equal to 0) or 1 (rainy -- otherwise).
2. Modify the functions from previous exercise so the fprop implements `sigmoid` on top of affine transform.
3. Modify cost function to binary cross-entropy
4. Make sure you compute the gradients correctly (as you have changed both the output and the cost)
